# Chatbot Educativo Interactivo
Proyecto adaptado con TF-IDF, Naive Bayes, similitud de coseno y ventana gráfica (Tkinter)

## 1. Importación de Librerías

In [1]:

import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import spacy
import tkinter as tk
from tkinter import ttk
import threading
import time
import unicodedata
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import make_pipeline

nlp = spacy.load("es_core_news_md")

## 2. Definición del Dataset

In [2]:
## 📥 Cargar el Dataset
df = pd.read_csv(r"C:\Users\darly\OneDrive\Escritorio\materialClaseIA\claseFIN8\dataEducacion_final_393_agregado_2.csv")

df.head()

,consulta,categoria,respuesta,respuestas_compuestas,consulta_
0,¿Cómo me inscribo a clases el próximo semestre?,matriculas,"Para inscribirte al próximo semestre, debes in...",¿Cómo me inscribo a clases el próximo semestre...,¿Cómo me inscribo a clases el próximo semestre...
1,¿Cuál es la fecha límite para inscribirse?,matriculas,La fecha límite para completar tu inscripción ...,¿Cuál es la fecha límite para inscribirse? La ...,¿Cuál es la fecha límite para inscribirse? mat...
2,¿Qué documentos necesito para la matrícula?,matriculas,Para realizar tu matrícula necesitas: identifi...,¿Qué documentos necesito para la matrícula? Pa...,¿Qué documentos necesito para la matrícula? ma...
3,¿Cuánto cuesta la inscripción?,matriculas,El costo de inscripción para el semestre actua...,¿Cuánto cuesta la inscripción? El costo de ins...,¿Cuánto cuesta la inscripción? matriculas
4,¿Dónde puedo pagar mi matrícula?,matriculas,Puedes pagar tu matrícula en cualquier sucursa...,¿Dónde puedo pagar mi matrícula? Puedes pagar ...,¿Dónde puedo pagar mi matrícula? matriculas


## 3. Preprocesamiento y limpieza 

In [3]:
#limpieza de los datos #Incluye:
#- Minúsculas
#- Eliminación de tildes
#- Lematización
#- Conserva palabras importantes como "dónde", "cuándo", "cómo", "qué", etc.

# Palabras que queremos conservar (normalizadas)
custom_stopwords_to_keep = {"no", "si", "donde", "cuando", "como", "que", "cual", "cuanto"}

def normalize_text(text):
    # Paso 1: Convertir a minúsculas
    text = text.lower()

    # Paso 2: Eliminar tildes y ñ
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore').decode('utf-8')  # elimina tildes y ñ

    # Paso 3: Eliminar signos de puntuación
    text = re.sub(r"[^\w\s]", "", text)

    # Paso 4: Eliminar espacios extra
    text = re.sub(r"\s+", " ", text).strip()

    # Paso 5: Procesar con spaCy
    doc = nlp(text)

    # Paso 6: Lematizar, eliminar stopwords y quitar tildes también a los lemas
    tokens = [
        ''.join(
            c for c in unicodedata.normalize('NFD', token.lemma_)
            if unicodedata.category(c) != 'Mn'
        )
        for token in doc
        if not (token.is_stop and token.lemma_.lower() not in custom_stopwords_to_keep)
    ]

    return " ".join(tokens)


In [4]:
# Aplicar limpieza
df["consulta_limpia"] = df["consulta"].apply(normalize_text)

In [5]:
df

,consulta,categoria,respuesta,respuestas_compuestas,consulta_,consulta_limpia
0,¿Cómo me inscribo a clases el próximo semestre?,matriculas,"Para inscribirte al próximo semestre, debes in...",¿Cómo me inscribo a clases el próximo semestre...,¿Cómo me inscribo a clases el próximo semestre...,como inscribo clase semestre
1,¿Cuál es la fecha límite para inscribirse?,matriculas,La fecha límite para completar tu inscripción ...,¿Cuál es la fecha límite para inscribirse? La ...,¿Cuál es la fecha límite para inscribirse? mat...,cual fecha limite inscribir el
2,¿Qué documentos necesito para la matrícula?,matriculas,Para realizar tu matrícula necesitas: identifi...,¿Qué documentos necesito para la matrícula? Pa...,¿Qué documentos necesito para la matrícula? ma...,que documento necesitar matricula
3,¿Cuánto cuesta la inscripción?,matriculas,El costo de inscripción para el semestre actua...,¿Cuánto cuesta la inscripción? El costo de ins...,¿Cuánto cuesta la inscripción? matriculas,cuanto costar inscripcion
4,¿Dónde puedo pagar mi matrícula?,matriculas,Puedes pagar tu matrícula en cualquier sucursa...,¿Dónde puedo pagar mi matrícula? Puedes pagar ...,¿Dónde puedo pagar mi matrícula? matriculas,donde pagar matricula
...,...,...,...,...,...,...
388,¿Se puede anular una calificación registrada p...,calificaciones,"Sí, si el docente lo justifica y se presenta e...",¿Se puede anular una calificación registrada p...,¿Se puede anular una calificación registrada p...,anular calificacion registrado error
389,¿Puedo ver el detalle de cómo se calculó mi no...,calificaciones,"Sí, puedes solicitar el desglose de calificaci...",¿Puedo ver el detalle de cómo se calculó mi no...,¿Puedo ver el detalle de cómo se calculó mi no...,detalle como calcular nota
390,¿Cuáles son las principales instalaciones de l...,instalaciones,Las principales instalaciones de la universida...,¿Cuáles son las principales instalaciones de l...,¿Cuáles son las principales instalaciones de l...,cual principal instalacion universidad
391,¿Cuáles son los principales requisitos que exi...,requisitos,Los principales requisitos que exige la univer...,¿Cuáles son los principales requisitos que exi...,¿Cuáles son los principales requisitos que exi...,cual principal requisito que exigir universida...


In [6]:
#Vamos a crear un modelo de clasificación de intención por categoría usando n-gramas de 1 a 3.

X = df["consulta_limpia"]
y = df["categoria"]

## 4. Vectorización TF-IDF

In [7]:
#partir set 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
#crear estructura de entranamiento y medir la clasificacion

model = make_pipeline(
    TfidfVectorizer(ngram_range=(1, 2),# Usa un contexto de hasta 3 palabras
                    min_df=1),           # Aparece en al menos 2 consulta
    MultinomialNB()
)


## 4. Entrenamiento del Modelo Naive Bayes

In [9]:
#entrenar modelo
model.fit(X_train, y_train)
print("Precisión del modelo:", model.score(X_test, y_test))
preds = model.predict(X_test)
print(classification_report(y_test, preds))


Precisión del modelo: 0.8481012658227848
                precision    recall  f1-score   support

calificaciones       0.93      0.93      0.93        14
      horarios       0.76      0.93      0.84        14
 instalaciones       1.00      0.92      0.96        12
    matriculas       0.82      0.82      0.82        11
    requisitos       0.79      0.79      0.79        14
      tramites       0.83      0.71      0.77        14

      accuracy                           0.85        79
     macro avg       0.86      0.85      0.85        79
  weighted avg       0.85      0.85      0.85        79



## 5. Función para Predecir y Responder con función de similitud

In [10]:
## 💬 Motor de Chatbot
#Definimos saludos y despedidas y usamos clasificación + similitud para elegir la mejor respuesta en cada categoría.

saludos = ["hola", "buenos dias", "buenas tardes", "buenas noches", "hola que tal", "como estas", "ey"]
respuestas_saludo = ["¡Hola! ¿En qué puedo ayudarte?", "¡Buenos días! ¿Qué necesitas?", "Hola, dime tu duda."]

despedidas = ["gracias", "hasta luego", "adios", "nos vemos", "bye", "chao"]
respuestas_despedida = ["¡Hasta pronto!", "Gracias por tu consulta. ¡Éxitos!", "Nos vemos."]

def responder(pregunta):
    pregunta_limpia = normalize_text(pregunta)
    print("🔍 Consulta limpia →", pregunta_limpia)
    
    if any(saludo in pregunta.lower() for saludo in saludos):
        return "saludo",random.choice(respuestas_saludo)
    
    if any(despedida in pregunta.lower() for despedida in despedidas):
        return "despedida", random.choice(respuestas_despedida)
    
    # 1. Clasificar la intención (etiqueta)
    etiqueta = model.predict([pregunta_limpia])[0]
    print("📌 Categoría detectada →", etiqueta)
    
    # 2. Filtrar respuestas de esa categoría
    respuestas_categoria = df[df["categoria"] == etiqueta].copy()

    # 3. Normalizar las respuestas para compararlas
    respuestas_categoria["respuesta_limpia"] = respuestas_categoria["respuestas_compuestas"].apply(normalize_text)

    # 4. Vectorizar respuestas limpias
    vectorizer = model.named_steps["tfidfvectorizer"]
    respuestas_categoria["respuesta_vec"] = respuestas_categoria["respuesta_limpia"].apply(lambda x: vectorizer.transform([x]))
    
    pregunta_vec = vectorizer.transform([pregunta_limpia])
    
    # 5. Calcular similitud contra respuestas
    respuestas_categoria["similitud"] = respuestas_categoria["respuesta_vec"].apply(lambda x: cosine_similarity(x, pregunta_vec)[0][0])
    
    # 6. Mostrar top 5 respuestas más similares
    print("\n🎯 Top 5 respuestas más similares:")
    top5 = respuestas_categoria[["respuestas_compuestas", "similitud"]].sort_values(by="similitud", ascending=False).head(5)
    print(top5.to_string(index=False))

    # 7. Devolver la mejor respuesta
    mejor_idx = respuestas_categoria["similitud"].idxmax()
    
    return etiqueta,df.loc[mejor_idx, "respuesta"]


## 6. Creación de la Interfaz Gráfica

In [11]:
# Crear ventana principal
ventana = tk.Tk()
ventana.title("Chat Educativo")
ventana.geometry("550x700")

# Frame de conversación
frame_conversacion = tk.Frame(ventana, bd=2, relief=tk.GROOVE)
frame_conversacion.pack(padx=15, pady=(15,5), fill=tk.BOTH, expand=True)

# Scrollbar y área de texto
scrollbar = tk.Scrollbar(frame_conversacion)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

texto_conversacion = tk.Text(frame_conversacion, wrap=tk.WORD, yscrollcommand=scrollbar.set, font=("Arial", 12))
texto_conversacion.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
scrollbar.config(command=texto_conversacion.yview)

# Configurar colores (tags)
texto_conversacion.tag_configure("usuario", foreground="black")
texto_conversacion.tag_configure("bot", foreground="blue")

# Barra de progreso
progress = ttk.Progressbar(ventana, mode='indeterminate')
progress.pack(padx=15, pady=(5,5), fill=tk.X)
progress.pack_forget()

# Frame de entrada
frame_entrada = tk.Frame(ventana)
frame_entrada.pack(padx=15, pady=(10,15), fill=tk.X)

# Entrada de texto
entrada_texto = tk.Entry(frame_entrada, font=("Arial", 14))
entrada_texto.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=(0,10))

# Botón de enviar
boton_enviar = tk.Button(frame_entrada, text="Enviar", font=("Arial", 12, "bold"), command=lambda: enviar_pregunta())
boton_enviar.pack(side=tk.RIGHT)

# Función para enviar pregunta
def enviar_pregunta():
    pregunta = entrada_texto.get()
    if pregunta.strip() == "":
        return
    texto_conversacion.insert(tk.END, f"👤 Tú: {pregunta}\n\n", "usuario")  # Se aplica el estilo "usuario"
    texto_conversacion.see(tk.END)
    entrada_texto.delete(0, tk.END)
    progress.pack(padx=15, pady=(5,5), fill=tk.X)
    progress.start()
    threading.Thread(target=procesar_pregunta, args=(pregunta,)).start()

# Función para procesar la respuesta
def procesar_pregunta(pregunta_usuario):
    time.sleep(1)
    categoria_predicha, respuesta = responder(pregunta_usuario)
    progress.stop()
    progress.pack_forget()
    texto_conversacion.insert(tk.END, f"🤖 Bot [{categoria_predicha}]: {respuesta}\n\n", "bot")  # Se aplica el estilo "bot"
    texto_conversacion.see(tk.END)

# Activar evento de Enter para enviar
ventana.bind('<Return>', lambda event: enviar_pregunta())

# Iniciar la ventana
ventana.mainloop()


🔍 Consulta limpia → hola
🔍 Consulta limpia → cual instalacion universidad
📌 Categoría detectada → instalaciones

🎯 Top 5 respuestas más similares:
                                                                                                                                                                                                                       respuestas_compuestas  similitud
                             ¿Cuáles son las principales instalaciones de la universidad? Las principales instalaciones de la universidad incluyen bibliotecas, laboratorios, salas de cómputo, canchas deportivas, cafeterías y auditorios.   0.611201
¿Qué servicios tecnológicos ofrece la universidad a los estudiantes? La universidad ofrece acceso a Wi-Fi en todo el campus, salas de cómputo, plataformas virtuales, correo institucional y licencias educativas de software especializado.   0.192060
                                                                                                  ¿Cómo acced